In [1]:
#Basic Imports
import pandas as pd
import os

#URL Image Download
import requests

#Google Cloud
from google.cloud import storage

In [2]:
# Save the excel file as a DataFrame to 'data'
file_path = 'directory_consumer_grade_images.xlsx'
data = pd.read_excel(file_path)
data.tail()

,NDC11,Part,Image,Layout,Name
133769,99207046730,1,PillProjectDisc103/images/TY5OVXLLOXV6H4I1TDVT...,MC_COOKED_CALIBRATED_V1.2,SOLODYN 105 MG TAB
133770,99207046730,1,PillProjectDisc31/images/BEIR3XK38EMGSDOZTWMUK...,MC_COOKED_CALIBRATED_V1.2,SOLODYN 105 MG TAB
133771,99207046730,1,PillProjectDisc69/images/CLJ1W40OS0XG5H6IVYT!N...,MC_COOKED_CALIBRATED_V1.2,SOLODYN 105 MG TAB
133772,99207046730,1,PillProjectDisc77/images/CSUHWDZ!XAZSEJHDANMFR...,MC_COOKED_CALIBRATED_V1.2,SOLODYN 105 MG TAB
133773,99207046730,1,PillProjectDisc23/images/B7P0-2BHT7T88IHB4E1EE...,MC_SPL_SPLIMAGE_V3.0,SOLODYN 105 MG TAB


In [3]:
#Checking the number of images per unique NDC11 ID which is the identifier for each pill.
data['NDC11'].value_counts()

NDC11
555901258      142
555903270      142
63459070160    136
52544029128    136
39022310       117
              ... 
78045805        14
173063302       12
173079102       12
173078302       12
49884041301     10
Name: count, Length: 4864, dtype: int64

In [4]:
# Only taking the top five pills with the most images.
#data = data[(data['NDC11'] == 49884041301)] # Only 10 images for testing!

data = data[(data['NDC11'] == 555901258) | (data['NDC11'] == 555903270) | (data['NDC11'] == 63459070160) | (data['NDC11'] == 52544029128) | (data['NDC11'] == 39022310)]
print(f'The total number of images is: {len(data)}')

The total number of images is: 673


In [5]:
# Setting up all the urls from which to download the images as a pandas Series.
pre_url = 'https://data.lhncbc.nlm.nih.gov/public/Pills/'
post_url = data['Image'] 
full_url = pre_url + post_url

In [6]:
LOCAL_DATA_PATH = os.path.join(os.path.expanduser('~'), "Downloads", "images")
key_file_path = os.path.join(os.path.expanduser('~'), ".pill-pic", "pill-pic-a4bcd429b85c.json") # Add the gcloud service acc JSON key to a .pill-pic folder.
BUCKET_NAME = "pill_pic_image_set"
counter = 1

client = storage.Client.from_service_account_json(key_file_path)
bucket = client.get_bucket(BUCKET_NAME)

for i, url in full_url.items():
    
    image_path = os.path.join(LOCAL_DATA_PATH, f"{i}.jpg")
    
    # Download the image from url and save locally.
    response = requests.get(url)
    with open(image_path, 'wb') as file:
        file.write(response.content)
    print(f"{i}.jpg saved to {LOCAL_DATA_PATH}.")
    
    # Upload the image to Google Cloud.
    blob = bucket.blob(f"{i}.jpg")
    blob.upload_from_filename(image_path)
    print(f"{i}.jpg uploaded to {BUCKET_NAME} on Google Cloud. \n ✅ {counter}/{len(data)}")
    counter += 1


4010.jpg saved to /Users/ninaadkalla/Downloads/images.
4010.jpg uploaded to pill_pic_image_set on Google Cloud. 
 ✅ 1/673
4011.jpg saved to /Users/ninaadkalla/Downloads/images.
4011.jpg uploaded to pill_pic_image_set on Google Cloud. 
 ✅ 2/673
4012.jpg saved to /Users/ninaadkalla/Downloads/images.
4012.jpg uploaded to pill_pic_image_set on Google Cloud. 
 ✅ 3/673
4013.jpg saved to /Users/ninaadkalla/Downloads/images.
4013.jpg uploaded to pill_pic_image_set on Google Cloud. 
 ✅ 4/673
4014.jpg saved to /Users/ninaadkalla/Downloads/images.
4014.jpg uploaded to pill_pic_image_set on Google Cloud. 
 ✅ 5/673
4015.jpg saved to /Users/ninaadkalla/Downloads/images.
4015.jpg uploaded to pill_pic_image_set on Google Cloud. 
 ✅ 6/673
4016.jpg saved to /Users/ninaadkalla/Downloads/images.
4016.jpg uploaded to pill_pic_image_set on Google Cloud. 
 ✅ 7/673
4017.jpg saved to /Users/ninaadkalla/Downloads/images.
4017.jpg uploaded to pill_pic_image_set on Google Cloud. 
 ✅ 8/673
4018.jpg saved to /Users